# Valuation of a Company

If you have watched Shark Tank, the judges try to offer deals on a certain valuation based on different parameters. Revenue, Gross Profit, Net Profit, Quarter on Quarter are some of these parameters. This got me wondering how do these judges calculate what valuation should be given using these parameters. Which parameter mattered the most? Which parameter would hold more weightage in a particular sector? How do you know that a certain valuation was a fair valuation?

So I decided a to build a Multiple Linear Regression Model to predict the valuation of companies belonging to a particular sector (to be precise of Sugar Sector). Then this Linear Regression Model can help me tell which companies are overbought and which companies are underbought.

Because I have good knowledge of the Stock Market, I compiled all the data in an excel sheet first, used the required formulas to get the necessary ratios and then converted it into a .csv file so that I can do the necessary analysis, pre-processing and build my Linear Regression Model.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
import piplite
await piplite.install('seaborn')
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sugar = pd.read_csv('Sugar_Stocks.csv')
sugar.head()

### Cleaning

I can see that there are some missing values in this Dataset. Let us clean them first.
Then I can see that there are some columns that do not have the appropriate data type due to formatting. So I will format them and assign them the appropriate data types.

In [ ]:
sugar = sugar.dropna()

In [ ]:
sugar['NoS'] = sugar['NoS'].str.replace(',', '').astype(int)

In [ ]:
sugar['Revenue'] = sugar['Revenue'].str.replace(',', '').astype(float)

In [ ]:
sugar['BV'] = sugar['BV'].str.replace(',', '').astype(float)

In [ ]:
sugar['EBIT CAGR 4 yrs'] = sugar['EBIT CAGR 4 yrs'].str.replace(',', '').astype(float)

In [ ]:
sugar['EBIT CAGR 2 yrs'] = sugar['EBIT CAGR 2 yrs'].str.replace(',', '').astype(float)

Let's check the data types before moving forward.

In [ ]:
sugar.dtypes

Because we want to make a valuation model, it would be fair to introduce a new column called for Market Capital. For those of you who don't know, Market Capital is just another name for valuation. The formula for Market Cap is Number of Shares of a company multiplied by its Last Trading Price.

In [ ]:
sugar['mcap'] = sugar['NoS'] * sugar['LTP']

In [ ]:
sugar.head()

Because Linear Regression Models are sensitive to outliers, we will check for any outliers in particular two important parameter. We will then clean our dataset by removing these outliers.

Let us first explore outliers in ROCE column (Return On Capital Equity).

In [ ]:
sns.boxplot(x=sugar['ROCE'])
plt.show()

Let us use the inter-quartile range method to deal with the outliers in this column.

In [ ]:
quantile_1 = sugar['ROCE'].quantile(0.25)
quantile_3 = sugar['ROCE'].quantile(0.75)
iqr = quantile_3 - quantile_1
upper_limit = quantile_3 + 1.5 * iqr
lower_limit = quantile_1 - 1.5 * iqr
sugar = sugar[(sugar['ROCE'] >= lower_limit) & (sugar['ROCE'] <= upper_limit)]
sugar = sugar[sugar['ROCE'] > 0]

Let us check if our outliers were removed

In [ ]:
sns.boxplot(x=sugar['ROCE'])
plt.show()

Let us now check outliers in the 'Debt to Equity Ratio' column.

In [ ]:
sns.boxplot(x=sugar['Debt to Equity Ratio'])
plt.show()

In [ ]:
quantile_1 = sugar['Debt to Equity Ratio'].quantile(0.25)
quantile_3 = sugar['Debt to Equity Ratio'].quantile(0.75)
iqr = quantile_3 - quantile_1
upper_limit = quantile_3 + 1.5 * iqr
lower_limit = quantile_1 - 1.5 * iqr
sugar = sugar[(sugar['Debt to Equity Ratio'] >= lower_limit) & (sugar['Debt to Equity Ratio'] <= upper_limit)]

Again, let us verify if any outliers are left or not.

In [ ]:
sns.boxplot(x=sugar['Debt to Equity Ratio'])
plt.show()

## Model Assumptions

Before we jump into building model, we first have to meet some criterias. These criterias or assumptions are necessary to build an effective model. These assumptions are:
* Independent Observations: All the observations should be independent of each other. We can assume this assumption to be true because no one company is related to other company.
* Linearity: To check which variables are linearly related to our target variable. We also need to check how much the independent variables are correlated to our dependent variable. This enables us to choose the right variables and filter out the unnecessary variables.
* No multi-colinearity: There should not be any linearity among any pair of independent variables that we choose.
* Normality: This assumption can only be checked after the model is built. The assumption of normality is met when the residuals are normally distributed.
* Homoscedasticity: The residuals should be scattered randomly around the fitted line. If the residuals are skewed to one or the other side then it means that there is some hidden correlation between the fitted values and predicted values.

### Linearity

Let us use the Pearson's Correlation method to measure the linear relationship between the independent variables and dependent variable (market cap)

In [ ]:
sugar.corr(method='pearson', numeric_only=True)[['mcap']].sort_values('mcap', ascending=False)

Seems like **Revenue** is most linearly related to the mcap. It is followed by **BV** (Book Value) and **R&S to SEC** (Reserves & Surplus to Share Equity Capital ratio). Ideally, a pearson correlation coefficient of 0.7 and above signifies a very high linear relationship. But to consider more factors, we will consider the variables with pearson correlation coefficient higher than 0.4. Due to this we will consider **ROA** (Return On Assets), **Revenue CQGR 5 qtrs** (compounded quarterly growth rate of revenue from the past 5 quarters), and **ROCE** (Return On Capital Equity).

### No multi-colinearity

We will check for linearity among the independent variables. If we detect a pair with high linearity among independent variables, we will drop the variable which has lower linearity with our dependent variable.

In [ ]:
sugar[['Revenue', 'BV', 'R&S to SEC', 'ROA', 'Revenue CQGR 5 yrs', 'ROCE', 'mcap']].corr(method='pearson')

There is high linearity between **Revenue** and **BV**. So we will have to choose 1 among them. Because **Revenue** and **mcap** has high correlation between them as compared to **BV** and **mcap**, we will drop **BV**.<br>
Due to same reasons we will have to drop **R&S to SEC** too.
Similar is the case between **ROA** and **ROCE**. So we will drop **ROCE**.
Although, there is high correlation between **Revenue CQGR 5 qtrs** and **ROA**, we will not drop either of the two and try to build model with both of them.

Using pairplot to visualize relationship between the independent variables and dependent variable.

In [ ]:
sns.pairplot(data=sugar[['Revenue', 'ROA', 'Revenue CQGR 5 yrs', 'mcap']])
plt.show()

In [ ]:
await piplite.install('statsmodels')
from statsmodels.formula.api import ols

The OLS formula doesn't accept spaces between the independent variables, so we will have to rename the **Revenue CQGR 5 yrs** column to **Revenue_CQGR_5_qtrs**.

In [ ]:
sugar.rename(columns = {'Revenue CQGR 5 yrs' : 'Revenue_CQGR_5_qtrs'}, inplace=True)

In [ ]:
ols_formula = "mcap ~ Revenue + ROA + Revenue_CQGR_5_qtrs"
OLS = ols(data=sugar[['Revenue', 'ROA', 'Revenue_CQGR_5_qtrs', 'mcap']], formula=ols_formula)
model = OLS.fit()
model.summary()

Earlier, we had considered **ROA** as well as **Revenue_CQGR_5_qtrs** for building our model despite the correlation between them.<br>
Let us again build the model after dropping **ROA** and then check whether the adjusted R-squared has improved or not.

In [ ]:
ols_formula = "mcap ~ Revenue + Revenue_CQGR_5_qtrs"
OLS = ols(data=sugar[['Revenue', 'Revenue_CQGR_5_qtrs', 'mcap']], formula=ols_formula)
model = OLS.fit()
model.summary()

We can clearly see, although by a very less margin, the adjusted R-squared did improve after we dropped **ROA**.

In [ ]:
model.resid

### Normality

We will check whether the residuals are normally distributed or not.<br>
We can do this in 2 ways:
* Plotting histogram of residuals
* Plotting qqplot of the residuals

We will use both.

In [ ]:
sns.histplot(model.resid)
plt.show()

In [ ]:
await piplite.install('statsmodels')
import statsmodels.api as sm
sm.qqplot(model.resid)

The normality assumption could be met only partially. But that could also be because we have very less sample size.

Let us now consider all the necessary columns and concatenate their respective residual values, and then arrange this list according to the residuals in ascending order.

In [ ]:
final = pd.concat([sugar[['Stock', 'Revenue', 'Revenue_CQGR_5_qtrs', 'mcap']], model.resid], axis=1)

In [ ]:
final.rename(columns = {0 : 'Residuals'}, inplace=True)

In [ ]:
final.sort_values(by=['Residuals', 'Revenue'], ascending=[True, False])

Generally, **P/E** ratio was used to check whether a company is overbought or underbought. But with this model, we will actually come to know which of the companies are overbought and which are underbought. We will come to know how good of an indicator **P/E** is.

In [ ]:
final = pd.concat([final, sugar['P/E']], axis=1)

In [ ]:
final = final.reset_index(drop=True)

In [ ]:
final.sort_values(by=['Residuals'])

We can observe that AVADHSUGAR, RANASUG and UTTAMSUGAR are most underbought stocks.<br>
We can understand why AVADHSUGAR and RANASUG are underbought. It might be because both of them have a negative quarter on quarter growth rate for last 5 quarters for revenue. The selling in these stocks might have caused to build a downward momentum which might have caused them to be in underbought zone. <br>
But UTTAMSUGAR checks all the points in our list. UTTAMSUGAR can be a safe bet.

In [ ]:
import statsmodels.api as sm

fig = plt.figure(figsize=(14, 8))
fig = sm.graphics.plot_regress_exog(model, 'Revenue', fig=fig)

In [ ]:
fig = plt.figure(figsize=(14, 8))
fig = sm.graphics.plot_regress_exog(model, 'Revenue_CQGR_5_qtrs', fig=fig)

### Homoscedasticity

We want to check whether the residuals are randomly scattered when plotted against the fitted values.

In [ ]:
X = sugar[['Revenue', 'Revenue_CQGR_5_qtrs']]
fitted_values = model.predict(X)

plt.figure(figsize=(12,8))
sns.scatterplot(y=model.resid, x=fitted_values)
plt.axhline(y=0)

plt.show()

We can see that the homoscedasticity assumption is met.

Let us plot a Revenue vs Residuals scatterplot to get a visual idea of which stocks are underbought and overbought.

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data=final, y='Residuals', x='Revenue')
plt.axhline(y=0)

for i in range(len(final)):
    plt.annotate(final['Stock'][i], (final['Revenue'][i]+50, final['Residuals'][i]-300000000))

plt.show()